In [12]:
# @title ###### Licensed to the Apache Software Foundation (ASF), Version 2.0 (the "License")

# Licensed to the Apache Software Foundation (ASF) under one
# or more contributor license agreements. See the NOTICE file
# distributed with this work for additional information
# regarding copyright ownership. The ASF licenses this file
# to you under the Apache License, Version 2.0 (the
# "License"); you may not use this file except in compliance
# with the License. You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing,
# software distributed under the License is distributed on an
# "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
# KIND, either express or implied. See the License for the
# specific language governing permissions and limitations
# under the License.

# 🌦️ Weather forecasting -- _Overview_

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/GoogleCloudPlatform/python-docs-samples/blob/main/people-and-planet-ai/weather-forecasting/notebooks/1-overview.ipynb)

This sample is broken into the following notebooks:

* ![Open in Colab](https://github.com/googlecolab/open_in_colab/raw/main/images/icon16.png) **🧭 Overview**:
  Go through what we want to achieve, and explore the data we want to use as _inputs and outputs_ for our model.

* [![Open in Colab](https://github.com/googlecolab/open_in_colab/raw/main/images/icon16.png) **🗄️ Create the dataset**](https://colab.research.google.com/github/GoogleCloudPlatform/python-docs-samples/blob/main/people-and-planet-ai/weather-forecasting/notebooks/2-dataset.ipynb):
  Use [Apache Beam](https://beam.apache.org/)
  to fetch data from [Earth Engine](https://earthengine.google.com/) in parallel, and create a dataset for our model in [Dataflow](https://cloud.google.com/dataflow).

* [![Open in Colab](https://github.com/googlecolab/open_in_colab/raw/main/images/icon16.png) **🧠 Train the model**](https://colab.research.google.com/github/GoogleCloudPlatform/python-docs-samples/blob/main/people-and-planet-ai/weather-forecasting/notebooks/3-training.ipynb):
  Build a simple _Fully Convolutional Network_ in [PyTorch](https://pytorch.org/) and train it in [Vertex AI](https://cloud.google.com/vertex-ai/docs/training/custom-training) with the dataset we created.

* [![Open in Colab](https://github.com/googlecolab/open_in_colab/raw/main/images/icon16.png) **🔮 Model predictions**](https://colab.research.google.com/github/GoogleCloudPlatform/python-docs-samples/blob/main/people-and-planet-ai/weather-forecasting/notebooks/4-predictions.ipynb):
  Get predictions from the model with data it has never seen before.

This sample leverages geospatial satellite and precipitation data from [Google Earth Engine](https://earthengine.google.com/).
Using satellite imagery, you'll build and train a model for rain "nowcasting" i.e. predicting the amount of rainfall for a given geospatial region and time in the immediate future.

* ⏲️ **Time estimate**: ~5 minutes
* 💰 **Cost estimate**: _free_

💚 This is one of many **machine learning how-to samples** inspired from **real climate solutions** aired on the [People and Planet AI 🎥 series](https://www.youtube.com/playlist?list=PLIivdWyY5sqI-llB35Dcb187ZG155Rs_7).

## 📒 Using this interactive notebook

Click the **run** icons ▶️ of each section within this notebook.

![Run cell](https://github.com/GoogleCloudPlatform/python-docs-samples/blob/main/people-and-planet-ai/weather-forecasting/notebooks/images/run-cell.png?raw=1)

> 💡 Alternatively, you can run the currently selected cell with `Ctrl + Enter` (or `⌘ + Enter` in a Mac).

This **notebook code lets you train and deploy an ML model** from end-to-end. When you run a code cell, the code runs in the notebook's runtime, so you're not making any changes to your personal computer.

> ⚠️ **To avoid any errors**, wait for each section to finish in their order before clicking the next “run” icon.

This sample must be connected to a **Google Cloud project**, but nothing else is needed other than your Google Cloud project.

You can use an _existing project_. Alternatively, you can create a new Cloud project [with cloud credits for free.](https://cloud.google.com/free/docs/gcp-free-tier)

# 🎬 Before you begin

Let's start by cloning the GitHub repository, and installing some dependencies.

In [13]:
# Now let's get the code from GitHub and navigate to the sample.
!git clone https://github.com/GoogleCloudPlatform/python-docs-samples.git
%cd python-docs-samples/people-and-planet-ai/weather-forecasting

fatal: destination path 'python-docs-samples' already exists and is not an empty directory.
/content/python-docs-samples/people-and-planet-ai/weather-forecasting/python-docs-samples/people-and-planet-ai/weather-forecasting


In [14]:
# Upgrade `setuptools` to install packages from pyproject.toml files.
!pip install --quiet --upgrade --no-warn-conflicts pip setuptools

# Install the `weather-data` local package.
!pip install serving/weather-data

Processing ./serving/weather-data
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for weather-data: filename=weather_data-1.0.0-py3-none-any.whl size=3895 sha256=8a31c8453a46c4e2ffce1f68432ecde290ac347282a3308cf3677b8994e8a3d4
  Stored in directory: /root/.cache/pip/wheels/a3/40/ea/3a7732d8a7e1e5ddab38c75544e930fb8af8a338fd9cc4c32a
Successfully built weather-data
  Attempting uninstall: weather-data
    Found existing installation: weather-data 1.0.0
    Uninstalling weather-data-1.0.0:
      Successfully uninstalled weather-data-1.0.0


## ☁️ My Google Cloud resources

Make sure you have followed these steps to configure your Google Cloud project:

1. Enable the APIs: _Earth Engine_

  <button>

  [Click here to enable the APIs](https://console.cloud.google.com/flows/enableapi?apiid=earthengine.googleapis.com)
  </button>

1. Register your
  [Compute Engine default service account](https://console.cloud.google.com/iam-admin/iam)
  on Earth Engine.

  <button>

  [Click here to register your service account on Earth Engine](https://signup.earthengine.google.com/#!/service_accounts)
  </button>

Once you have everything ready, you can go ahead and fill in your Google Cloud resources in the following code cell.
Make sure you run it!

In [15]:
from __future__ import annotations

import os
from google.colab import auth

# Please fill in these values.
project = "snappy-provider-456612-d8"  # @param {type:"string"}

# Quick input validations.
assert project, "snappy-provider-456612-d8"

# Authenticate to Colab.
auth.authenticate_user()

# Set GOOGLE_CLOUD_PROJECT for google.auth.default().
os.environ["GOOGLE_CLOUD_PROJECT"] = project

# Set the gcloud project for other gcloud commands.
!gcloud config set project {project}

Updated property [core/project].


# 🧭 Overview

The goal of our model is using satellite images to do _weather forecasting_.
Specifically, we want to predict the amount of rainfall, measured in millimeters per hour, for the next two to six hours in the future.
This kind of short term forecasting is called [weather _nowcasting_](https://en.wikipedia.org/wiki/Nowcasting_(meteorology)).

When working with satellite data, each image has the shape `(width, height, bands)`.
**Bands** contain _numeric values_ for each pixel in the image, like the measurements from specific satellite instruments for different ranges of the electromagnetic spectrum, or the probabilities of different classifications.
If you're familiar with image classification problems, you can think of the bands as similar to an image's RGB channels.

## ☔️ Precipitation

We use [NASA's Global Precipitation Measurement (GPM)](https://developers.google.com/earth-engine/datasets/catalog/NASA_GPM_L3_IMERG_V06) to get the amount of _precipitation_ of rain and snow, measured as millimeters per hour.
We're interested in the `precipitationCal` band, which gives us the _calibrated_ precipitation amount.

This is what we want to predict, so we'll use them for our _labels_.
But it's also useful for the model to look at the precipitation from the _past_, so we'll also use it as _inputs_.

In the [`serving/data.py`](../serving/data.py) file, we defined a function called `get_gpm_sequence` which returns us an `ee.Image` with the precipitation values for the time sequence we give it.
Each time step is stored in a different band with the index as a prefix.
For example, the band corresponding to the first time step in the sequence would be `0_precipitationCal`, and the second time step would be `1_precipitationCal`.

In [16]:
from datetime import datetime
import folium
import ee

from weather.data import get_gpm_sequence


def gpm_layer(image: ee.Image, label: str, i: int) -> folium.TileLayer:
    vis_params = {
        "bands": [f"{i}_precipitationCal"],
        "min": 0.0,
        "max": 20.0,
        "palette": [
            "000096",
            "0064ff",
            "00b4ff",
            "33db80",
            "9beb4a",
            "ffeb00",
            "ffb300",
            "ff6400",
            "eb1e00",
            "af0000",
        ],
    }
    # Mask (hide) pixels with no precipitation to see the map below.
    image = image.mask(image.gt(0.1))
    return folium.TileLayer(
        name=f"[{label}] Precipitation",
        tiles=image.getMapId(vis_params)["tile_fetcher"].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        overlay=True,
    )


# Get the Earth Engine images.
dates = [datetime(2019, 9, 2, 18)]
image = get_gpm_sequence(dates)

# Show map.
map = folium.Map([25, -90], zoom_start=5)
for i, date in enumerate(dates):
    gpm_layer(image, str(date), i).add_to(map)
folium.LayerControl().add_to(map)
map

![Global Precipitation Measurement (GPM)](https://github.com/GoogleCloudPlatform/python-docs-samples/blob/main/people-and-planet-ai/weather-forecasting/notebooks/images/gpm.png?raw=1)

> 💡 This is [Hurricane Dorian](https://en.wikipedia.org/wiki/Hurricane_Dorian), the strongest Category 5 hurricane on record in the Bahamas.

## 🌨 Cloud and moisture

To predict precipitation, it's also useful to take a look at the _cloud_ and _moisture_.
We use data from [GOES-16 Cloud and Moisture Imagery](https://developers.google.com/earth-engine/datasets/catalog/NOAA_GOES_16_MCMIPF), which was the first satellite from the [Geostationary Operational Environmental Satellites (GOES)](https://en.wikipedia.org/wiki/Geostationary_Operational_Environmental_Satellite) mission, operated by [NASA](https://en.wikipedia.org/wiki/NASA) and [NOAA](https://en.wikipedia.org/wiki/National_Oceanic_and_Atmospheric_Administration).
It includes measurements from the _visible_, _near-infrared_, and _infrared_ spectrum.
It is a [geostationary](https://en.wikipedia.org/wiki/Geostationary_orbit) satellite, so its orbit is synchronized with the Earth's rotation, and it provides a view centered in the Americas.

In the [`serving/data.py`](../serving/data.py) file, we defined a function called `get_goes16_sequence` which returns us an `ee.Image` with the cloud and moisture data for the time sequence we give it.

In [17]:
from datetime import datetime
import folium
import ee

from weather.data import get_goes16_sequence


def goes16_layer(image: ee.Image, label: str, i: int) -> folium.TileLayer:
    vis_params = {
        "bands": [f"{i}_CMI_C02", f"{i}_CMI_C03", f"{i}_CMI_C01"],
        "min": 0.0,
        "max": 3000.0,
    }
    return folium.TileLayer(
        name=f"[{label}] Cloud and moisture",
        tiles=image.getMapId(vis_params)["tile_fetcher"].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        overlay=True,
    )


# Get the Earth Engine image.
dates = [datetime(2019, 9, 2, 18)]
image = get_goes16_sequence(dates)

# Show map.
map = folium.Map([25, -90], zoom_start=5)
for i, date in enumerate(dates):
    goes16_layer(image, str(date), i).add_to(map)
folium.LayerControl().add_to(map)
map

![GOES 16](https://github.com/GoogleCloudPlatform/python-docs-samples/blob/main/people-and-planet-ai/weather-forecasting/notebooks/images/goes16.png?raw=1)

## 🏔 Elevation

Elevation could also give the model useful information.
We use the [MERIT Terrain DEM](https://developers.google.com/earth-engine/datasets/catalog/MERIT_DEM_v1_0_3) dataset to get the elvation.

In the [`serving/data.py`](../serving/data.py) file, we defined a function called `get_elevation` which returns us an `ee.Image` with the elevation measured in meters.

In [18]:
import folium

from weather.data import get_elevation


def elevation_layer() -> folium.TileLayer:
    image = get_elevation()
    vis_params = {
        "bands": ["elevation"],
        "min": 0.0,
        "max": 3000.0,
        "palette": [
            "000000",
            "478FCD",
            "86C58E",
            "AFC35E",
            "8F7131",
            "B78D4F",
            "E2B8A6",
            "FFFFFF",
        ],
    }
    return folium.TileLayer(
        name="Elevation",
        tiles=image.getMapId(vis_params)["tile_fetcher"].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        overlay=True,
    )


# Show map.
map = folium.Map([25, -90], zoom_start=5)
elevation_layer().add_to(map)
folium.LayerControl().add_to(map)
map

![Elevation](https://github.com/GoogleCloudPlatform/python-docs-samples/blob/main/people-and-planet-ai/weather-forecasting/notebooks/images/elevation.png?raw=1)

## 🛰 Inputs

In this example, we also consider multiple images across time, since weather forecasting is more accurate when we look at how the cloud cover changes over a period of time.
Particularly, we consider 3 data points -- 4 hours prior, 2 hours prior, and current.

> 💡 To give the model a better picture, we chose to feed it with _at least three_ data points from the past.
> With only a single point, the model wouldn't know if the rain is increasing or decreasing.
> Two points would give it an idea of the trend.
> Three or more points would give it an idea of how fast it's changing.
> The more points, the more it can see.

In the [`serving/data.py`](serving/data.py) file, we defined a function called `get_inputs_image` which returns us an `ee.Image` with bands for all the time steps for cloud and moisture, and for precipitation, alongside with the elevation.

In [19]:
from datetime import datetime, timedelta
import folium

from weather.data import get_inputs_image

# Get the Earth Engine image.
date = datetime(2019, 9, 2, 18)
image = get_inputs_image(date)

# Get 4 hours prior, 2 hours prior, and current time.
input_hour_deltas = [-4, -2, 0]

# Show map.
map = folium.Map([25, -90], zoom_start=5)
elevation_layer().add_to(map)
for i, h in enumerate(input_hour_deltas):
    label = str(date + timedelta(hours=h))
    goes16_layer(image, label, i).add_to(map)
    gpm_layer(image, label, i).add_to(map)
folium.LayerControl().add_to(map)
map

KeyboardInterrupt: 

![Inputs](https://github.com/GoogleCloudPlatform/python-docs-samples/blob/main/people-and-planet-ai/weather-forecasting/notebooks/images/inputs.png?raw=1)

> 💡 You can hide and show layers from the top-right corner widget to see all the inputs for the model.

## ✅ Labels

We chose to predict precipitation for 2 and 6 hours in the future, but it could be anything as long as we have the right _labels_.

In the [`serving/data.py`](../serving/data.py) file, we defined a function called `get_labels_image` which returns us an `ee.Image` with bands for each time step of precipitation.

In [ ]:
from datetime import datetime, timedelta
import folium

from weather.data import get_labels_image, OUTPUT_HOUR_DELTAS

# Get the Earth Engine image.
date = datetime(2019, 9, 3, 18)
image = get_labels_image(date)

# Show map.
map = folium.Map([25, -90], zoom_start=5)
for i, h in enumerate(OUTPUT_HOUR_DELTAS):
    label = str(date + timedelta(hours=h))
    gpm_layer(image, label, i).add_to(map)
folium.LayerControl().add_to(map)
map

![Labels](https://github.com/GoogleCloudPlatform/python-docs-samples/blob/main/people-and-planet-ai/weather-forecasting/notebooks/images/labels.png?raw=1)

# ⛳️ What's next?

* [![Open in Colab](https://github.com/googlecolab/open_in_colab/raw/main/images/icon16.png) **🗄️ Create the dataset**](https://colab.research.google.com/github/GoogleCloudPlatform/python-docs-samples/blob/main/people-and-planet-ai/weather-forecasting/notebooks/2-dataset.ipynb):
  Use [Apache Beam](https://beam.apache.org/)
  to fetch data from [Earth Engine](https://earthengine.google.com/) in parallel, and create a dataset for our model in [Dataflow](https://cloud.google.com/dataflow).

* [![Open in Colab](https://github.com/googlecolab/open_in_colab/raw/main/images/icon16.png) **🧠 Train the model**](https://colab.research.google.com/github/GoogleCloudPlatform/python-docs-samples/blob/main/people-and-planet-ai/weather-forecasting/notebooks/3-training.ipynb):
  Build a simple _Fully Convolutional Network_ in [PyTorch](https://pytorch.org/) and train it in [Vertex AI](https://cloud.google.com/vertex-ai/docs/training/custom-training) with the dataset we created.

* [![Open in Colab](https://github.com/googlecolab/open_in_colab/raw/main/images/icon16.png) **🔮 Model predictions**](https://colab.research.google.com/github/GoogleCloudPlatform/python-docs-samples/blob/main/people-and-planet-ai/weather-forecasting/notebooks/4-predictions.ipynb):
  Get predictions from the model with data it has never seen before.